# Módulo 07, Tarefa 01

Vamos começar a mexer na nossa base de projeto? Já fizemos um exercício de montar a variável resposta, já pudemos perceber que essa atividade pode não ser trivial. Vamos agora trabalhar a base para que fique propícia ao *scikitlearn* para trabalharmos.

Lembrando, a base se chama demo01.csv, e originalmente está publicada [aqui](https://www.kaggle.com/rikdifos/credit-card-approval-prediction).

#### 1) Carregue a base e avalie:

- As variáveis
- Tipos de dados de cada variável
- Quantidade de missings
- Distribuição da variável resposta (mau)

In [48]:
# Importar bibliotecas
import pandas as pd

# Carregar a base de dados
# Certifique-se de que o arquivo 'Profissão Cientista de Dados M7 A1 - demo01.csv'
# esteja no mesmo diretório do seu Jupyter Notebook, ou forneça o caminho completo.
try:
    df = pd.read_csv('Profissão Cientista de Dados M7 A1 - demo01.csv')
    print("Base de dados carregada com sucesso!")
except FileNotFoundError:
    print("ERRO: O arquivo 'Profissão Cientista de Dados M7 A1 - demo01.csv' não foi encontrado.")
    print("Por favor, verifique se o arquivo está no diretório correto ou se o nome está exato.")
    # Exiting or handling the error gracefully, for now, we'll just print and stop
    exit() # You might want to remove this line if you want the rest of the code to run with an empty df or handle it differently

# Limpar espaços nos nomes das colunas
df.columns = df.columns.str.strip()

# Verificar informações gerais
print("\nTipos de dados de cada variável:")
print(df.dtypes)

# Verificar quantidade de valores ausentes (missing)
print("\nQuantidade de missings por coluna:")
print(df.isnull().sum())

# Conferir se a coluna 'mau' existe
print("\nColunas da base:")
print(df.columns.tolist())

print("\nVerificando se 'mau' está na base:")
if 'mau' in df.columns:
    print("✅ A coluna 'mau' está na base.")
    print("\nValores únicos da variável 'mau':")
    print(df['mau'].unique())

    # Converter 'mau' de booleano para inteiro, se necessário
    # Esta conversão é importante para que a variável resposta seja numérica (0 ou 1)
    if df['mau'].dtype == 'bool':
        df['mau'] = df['mau'].astype(int)
        print("Coluna 'mau' convertida para tipo inteiro (0 ou 1).")
    elif df['mau'].dtype not in ['int64', 'float64']:
        # Se 'mau' não for bool, int ou float, tentar converter para int
        try:
            df['mau'] = df['mau'].astype(int)
            print("Coluna 'mau' convertida para tipo inteiro (0 ou 1).")
        except ValueError:
            print("AVISO: Não foi possível converter a coluna 'mau' para inteiro. Verifique seus valores.")
            # Se não puder converter, pode ser que a coluna contenha strings não numéricas
            print(df['mau'].value_counts())

    # Verificar a distribuição da variável resposta 'mau' após a conversão
    print("\nDistribuição da variável 'mau':")
    print(df['mau'].value_counts(normalize=True))
    print(df['mau'].value_counts())

else:
    print("❌ A coluna 'mau' não foi encontrada. Verifique se há erro no nome.")
    print("Por favor, corrija o nome da coluna ou o arquivo de dados.")
    # Se 'mau' não existe, as próximas etapas podem falhar. Considere adicionar um `exit()` ou tratamento mais robusto aqui.

Base de dados carregada com sucesso!

Tipos de dados de cada variável:
sexo                      object
posse_de_veiculo          object
posse_de_imovel           object
qtd_filhos                 int64
tipo_renda                object
educacao                  object
estado_civil              object
tipo_residencia           object
idade                    float64
tempo_emprego            float64
possui_celular             int64
possui_fone_comercial      int64
possui_fone                int64
possui_email               int64
qt_pessoas_residencia    float64
mau                         bool
dtype: object

Quantidade de missings por coluna:
sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
possui_celular           0
possui_fone_comercial    0
possui_fone              0


#### 2) Vamos montar um metadados

1. Crie um dataframe com os nomes de cada variável e o tipo de dados de cada variável.
2. Adicione uma coluna nesse *dataframe* chamada "qtd_categorias" e coloque nela o número de categorias correspondente de cada variável.
    Dica:
        1. inicie uma lista vazia
        2. faça um for ao longo dos nomes das variáveis,
        3. conte o número de categorias dessa variável
        4. acumule essa informação de 3. na lista que você criou em 1.
        5. No final, essa lista pode ser atribuída à nossa variável.

In [50]:
# Criar dataframe com nomes das variáveis e tipos
metadados = pd.DataFrame({
    'variavel': df.columns,
    'tipo_dado': df.dtypes.astype(str)
})

# Contar quantidade de categorias por variável
qtd_categorias = []

for col in df.columns:
    qtd = df[col].nunique()
    qtd_categorias.append(qtd)

# Adicionar essa informação no metadados
metadados['qtd_categorias'] = qtd_categorias

print("\nMetadados:")
print(metadados)


Metadados:
                                    variavel tipo_dado  qtd_categorias
sexo                                    sexo    object               2
posse_de_veiculo            posse_de_veiculo    object               2
posse_de_imovel              posse_de_imovel    object               2
qtd_filhos                        qtd_filhos     int64               8
tipo_renda                        tipo_renda    object               5
educacao                            educacao    object               5
estado_civil                    estado_civil    object               5
tipo_residencia              tipo_residencia    object               6
idade                                  idade   float64            5298
tempo_emprego                  tempo_emprego   float64            3005
possui_celular                possui_celular     int64               1
possui_fone_comercial  possui_fone_comercial     int64               2
possui_fone                      possui_fone     int64           

#### 3) Crie variáveis dummy para as variáveis necessárias (i.e. aquelas que são qualitativas e não estão armazenadas como {0, 1} ou {True, False}. Crie um *dataframe* apenas com as variáveis apropriadas para entrada no scikitlearn - elimine as variáveis tipo *str*, mantendo apenas suas versões *dummy*.

In [52]:
# Identificar colunas categóricas (tipo 'object' ou 'category') para criar dummies
# As colunas numéricas (int, float) e a variável 'mau' já convertida serão mantidas.
# Colunas booleanas que o pandas leu como 'bool' ou 'object' e que representam 0/1 ou True/False
# podem ser convertidas para int antes deste passo se for o caso, para não virarem dummies desnecessariamente.
# O pd.get_dummies por padrão ignora colunas numéricas, e transforma apenas as colunas 'object' ou 'category'.

# 'df_dummies' será o DataFrame com as variáveis dummy criadas e as numéricas mantidas.
# 'drop_first=True' evita a multicolinearidade, criando N-1 colunas para N categorias.
# 'dtype=int' garante que as novas colunas dummy sejam do tipo inteiro (0 ou 1).
df_dummies = pd.get_dummies(df, drop_first=True, dtype=int)

print("\nPrimeiras 5 linhas do DataFrame com variáveis dummy (df_dummies):")
print(df_dummies.head())
print("\nInformações do DataFrame com variáveis dummy (df_dummies):")
print(df_dummies.info())


Primeiras 5 linhas do DataFrame com variáveis dummy (df_dummies):
   qtd_filhos      idade  tempo_emprego  possui_celular  \
0           0  58.832877       3.106849               1   
1           0  52.356164       8.358904               1   
2           0  52.356164       8.358904               1   
3           0  46.224658       2.106849               1   
4           0  29.230137       3.021918               1   

   possui_fone_comercial  possui_fone  possui_email  qt_pessoas_residencia  \
0                      0            0             0                    2.0   
1                      0            1             1                    1.0   
2                      0            1             1                    1.0   
3                      1            1             1                    2.0   
4                      0            0             0                    2.0   

   mau  sexo_M  ...  educacao_Secondary / secondary special  \
0    0       1  ...                           

#### 4) Qual variável é mais poderosa?

Considere as variáveis ```possui_email``` e ```posse_de_veiculo```. Faça uma tabela cruzada entre elas e responda qual delas te parece mais poderosa para prever a probabilidade de ```mau = 1```?

In [54]:
# Tabelas cruzadas com normalização por linha (index)
# Isso mostra a proporção de 'mau=1' para cada categoria de 'possui_email' e 'posse_de_veiculo'.
if 'possui_email' in df.columns and 'posse_de_veiculo' in df.columns and 'mau' in df.columns:
    cross_email = pd.crosstab(df['possui_email'], df['mau'], normalize='index')
    cross_veiculo = pd.crosstab(df['posse_de_veiculo'], df['mau'], normalize='index')

    print("\nTabela cruzada - Probabilidade de mau=1 dado possui_email:")
    print(cross_email)

    print("\nTabela cruzada - Probabilidade de mau=1 dado posse_de_veiculo:")
    print(cross_veiculo)

    print("\nAnálise de poder preditivo:")
    print("Para determinar qual variável é mais poderosa, observe as diferenças na probabilidade de 'mau=1' entre as categorias.")
    print("Uma variável é mais poderosa se houver uma diferença maior na probabilidade de 'mau=1' entre suas categorias.")
    print("Por exemplo, se a probabilidade de 'mau=1' para quem 'possui_email=0' é 5% e para quem 'possui_email=1' é 10%,")
    print("e para 'posse_de_veiculo=0' é 6% e para 'posse_de_veiculo=1' é 7%, 'possui_email' seria mais poderosa.")
else:
    print("\nAVISO: As colunas 'possui_email', 'posse_de_veiculo' ou 'mau' não foram encontradas para a análise de poder preditivo.")


Tabela cruzada - Probabilidade de mau=1 dado possui_email:
mau                  0         1
possui_email                    
0             0.976269  0.023731
1             0.979730  0.020270

Tabela cruzada - Probabilidade de mau=1 dado posse_de_veiculo:
mau                      0         1
posse_de_veiculo                    
N                 0.975732  0.024268
Y                 0.977905  0.022095

Análise de poder preditivo:
Para determinar qual variável é mais poderosa, observe as diferenças na probabilidade de 'mau=1' entre as categorias.
Uma variável é mais poderosa se houver uma diferença maior na probabilidade de 'mau=1' entre suas categorias.
Por exemplo, se a probabilidade de 'mau=1' para quem 'possui_email=0' é 5% e para quem 'possui_email=1' é 10%,
e para 'posse_de_veiculo=0' é 6% e para 'posse_de_veiculo=1' é 7%, 'possui_email' seria mais poderosa.


#### 5) Salve a base, pois ela será utilizada no final deste módulo.

In [58]:
# Salvar a base pronta para machine learning
# Certifique-se de que 'df_dummies' foi criado na etapa anterior.
try:
    df_dummies.to_csv('demo01_tratado.csv', index=False)
    print("\nBase salva como 'demo01_tratado.csv'.")
except NameError:
    print("\nERRO: 'df_dummies' não foi definido. Verifique a execução da etapa 3.")
except Exception as e:
    print(f"\nOcorreu um erro ao salvar a base: {e}")


Base salva como 'demo01_tratado.csv'.
